In [ ]:
import os
import json
import pandas as pd

DEBUG = True  # Set to True to enable debugging output

# Mapping: keys are the splits (categories) and values are lists of role names.
ROLE_DATASET_MAPPING = {
    "econ": ["economic researcher", "economist", "financial analyst"],
    "eecs": ["electronics technician", "data scientist", "electrical engineer", "software engineer", "web developer"],
    "law": ["bailiff", "lawyer"],
    "math": ["data analyst", "mathematician", "statistician"],
    "medicine": ["nurse", "doctor", "physician", "dentist", "surgeon"],
    "natural_science": ["geneticist", "biologist", "physicist", "teacher", "chemist", "ecologist"],
    "politics": ["politician", "sheriff", "governor", "enthusiast", "partisan"],
    "psychology": ["psychologist"]
}

def debug_print(message):
    if DEBUG:
        print(message)

def get_category_from_role(role_name):
    for category, roles in ROLE_DATASET_MAPPING.items():
        if role_name in roles:
            debug_print(f"Role '{role_name}' belongs to category '{category}'")
            return category
    debug_print(f"Role '{role_name}' does not match any known category")
    return None

def collect_directions_from_split(split_dir):
    directions = set()
    debug_print(f"Collecting directions from split directory: {split_dir}")
    for coeff in ["1.0", "3.0"]:
        select_dir = os.path.join(split_dir, coeff, "select_direction")
        json_file = os.path.join(select_dir, "direction_evaluations_filtered.json")
        if os.path.isfile(json_file):
            debug_print(f"Found JSON file: {json_file}")
            try:
                with open(json_file, "r") as f:
                    data = json.load(f)
                    for item in data:
                        layer = item.get("layer")
                        position = item.get("position")
                        if layer is not None and position is not None:
                            directions.add((layer, position))
            except Exception as e:
                debug_print(f"Error reading {json_file}: {e}")
        else:
            debug_print(f"JSON file not found: {json_file}")
    debug_print(f"Collected directions: {directions}")
    return directions

def evaluate_direction(role_dir, direction):
    layer, position = direction
    filename = f"{layer}_{position}.json"
    test_file = os.path.join(role_dir, "test_direction", "3.0", filename)
    debug_print(f"Evaluating direction {direction} using test file: {test_file}")
    
    if not os.path.isfile(test_file):
        debug_print(f"Test file not found: {test_file}")
        return False

    try:
        with open(test_file, "r") as f:
            results = json.load(f)
            for res in results:
                passed_val = str(res.get("passed", "")).strip().lower()
                debug_print(f"In file {test_file}, found passed value: '{passed_val}'")
                if passed_val == "yes":
                    return True
    except Exception as e:
        debug_print(f"Error reading {test_file}: {e}")
    return False

def process_role_dir(role_dir, category):
    split_dir = os.path.join(role_dir, category)
    if not os.path.isdir(split_dir):
        debug_print(f"Split directory '{split_dir}' does not exist in role folder '{role_dir}'")
        return (0, 0)

    directions = collect_directions_from_split(split_dir)
    if not directions:
        debug_print(f"No directions found in split '{split_dir}' for role folder '{role_dir}'")
        return (0, 0)

    passed_count = 0
    total = len(directions)
    for direction in directions:
        if evaluate_direction(role_dir, direction):
            passed_count += 1

    debug_print(f"In role folder '{role_dir}' (split '{category}'): Passed {passed_count} out of {total} directions")
    return (passed_count, total)

def format_fraction(passed, total):
    if total == 0:
        return "N/A"
    else:
        percentage = (passed / total) * 100
        return f"{passed}/{total} ({percentage:.0f}%)"

def build_results_table_by_role(root_dir):
    # Build the initial dictionary: rows = roles, columns = models, cell value = (passed, total)
    results = {}
    debug_print(f"Scanning root directory for models: {root_dir}")
    for model_name in os.listdir(root_dir):
        model_path = os.path.join(root_dir, model_name)
        if not os.path.isdir(model_path):
            debug_print(f"Skipping non-directory item: {model_path}")
            continue
        debug_print(f"Processing model: {model_name}")

        for role_name in os.listdir(model_path):
            role_path = os.path.join(model_path, role_name)
            if not os.path.isdir(role_path):
                continue

            category = get_category_from_role(role_name)
            if category is None:
                debug_print(f"Skipping role folder '{role_name}' in model '{model_name}' (not in mapping)")
                continue

            result_tuple = process_role_dir(role_path, category)
            if role_name not in results:
                results[role_name] = {}
            results[role_name][model_name] = result_tuple

    # Create a DataFrame with rows as roles and columns as models.
    df = pd.DataFrame.from_dict(results, orient="index")

    # --- Compute row totals (over the original model columns) ---
    original_cols = list(df.columns)  # these are the model names
    row_totals = {}
    for role in df.index:
        row_passed = 0
        row_total = 0
        for col in original_cols:
            cell = df.loc[role, col]
            if cell is not None:
                row_passed += cell[0]
                row_total += cell[1]
        row_totals[role] = (row_passed, row_total)
    df["Total Models"] = pd.Series(row_totals)

    # --- Compute column totals (across all roles) ---
    col_totals = {}
    for col in df.columns:
        col_passed = 0
        col_total = 0
        # Sum only over the original roles (exclude the "Total" row which we haven't added yet)
        for role in df.index:
            cell = df.loc[role, col]
            if cell is not None:
                col_passed += cell[0]
                col_total += cell[1]
        col_totals[col] = (col_passed, col_total)

    # --- Append the "Total" row ---
    df.loc["Total Roles"] = pd.Series(col_totals)

    # --- Format every cell using format_fraction ---
    df = df.applymap(lambda cell: format_fraction(cell[0], cell[1]) if cell is not None else "N/A")
    return df

if __name__ == "__main__":
    # Change this to the path of your models directory.
    root_models_dir = r"C:\Users\user\Desktop\temp\rolevectors_results"  # <--- UPDATE THIS PATH
    debug_print("Starting analysis for model/role evaluation...")
    df = build_results_table_by_role(root_models_dir)

    # Reorder columns as specified and append "Total" at the end
    new_order = [
        'gemma-2-2b-it', 
        'gemma-2-9b-it',
        'Llama-3.2-1B-Instruct', 
        'Llama-3.2-3B-Instruct', 
        'Llama-3.1-8B-Instruct', 
        'Qwen-1_8B-Chat', 
        'Qwen-7B-Chat'
    ]
    new_order.append("Total Models")
    df = df.reindex(columns=new_order)

    print("Fraction and percentage of directions that passed for each model/role (with totals):")
    print(df)


In [ ]:
df

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
import re

def latex_blue_gradient(val, vmin=0, vmax=100):
    """
    Convert a cell value (formatted as "passed/total (XX.XX%)" or "N/A")
    into a LaTeX string that applies a blue background.
    
    The intensity of the blue is determined by the percentage inside the parentheses.
    If the cell is "N/A" or the percentage cannot be parsed, it is treated as 0.
    
    The function also replaces "%" with "\%" for proper LaTeX rendering.
    """
    try:
        if isinstance(val, str):
            # Handle "N/A" explicitly.
            if val.strip() == "N/A":
                numeric_val = 0.0
            else:
                # Look for a pattern like "(20.00%)" and extract the percentage.
                m = re.search(r'\(([\d.]+)%\)', val)
                if m:
                    numeric_val = float(m.group(1))
                else:
                    numeric_val = 0.0
        else:
            numeric_val = float(val)
    except Exception:
        numeric_val = 0.0

    # Normalize the percentage value between vmin and vmax.
    norm_val = (numeric_val - vmin) / (vmax - vmin)
    norm_val = np.clip(norm_val, 0, 1)

    # Map the normalized value to an intensity (0 to 50).
    intensity = int(norm_val * 50)
    
    # Build the LaTeX cell color command.
    color_cmd = f"\\cellcolor{{blue!{intensity}}}" if intensity > 0 else ""
    
    # Replace "%" with "\%" for LaTeX.
    if isinstance(val, str):
        val = val.replace('%', r'\%')
    
    return f"{color_cmd} {val}"

def style_df_to_latex(df, vmin=0, vmax=100):
    """
    Apply the blue gradient styling to every cell in the DataFrame,
    converting each cell to a LaTeX string that includes a background color.
    This function assumes that each cell is already formatted as a string
    like "passed/total (XX.XX%)" or "N/A".
    """
    df_styled = df.copy()
    for col in df.columns:
        df_styled[col] = df[col].apply(lambda x: latex_blue_gradient(x, vmin, vmax))
    return df_styled

# --- Usage Example ---

# For demonstration, suppose your DataFrame 'df' has cells such as:
# "1/5 (20.00%)", "3/4 (75.00%)", or "N/A".
# df = pd.DataFrame({
#     'Model A': ['1/5 (20.00%)', '3/4 (75.00%)', 'N/A'],
#     'Model B': ['2/5 (40.00%)', '4/4 (100.00%)', '1/5 (20.00%)']
# })

df_latex_styled = style_df_to_latex(df, vmin=0, vmax=100)

# Convert the styled DataFrame to LaTeX code.
# Note: escape=False is necessary so that our LaTeX commands (like \cellcolor) remain intact.
latex_code = df_latex_styled.to_latex(escape=False)
print(latex_code)
